In [63]:
import pandas as pd
import json

In [64]:
with open('coms-clean.json') as json_file:
    coms = json.load(json_file)
with open('evts-detail.json') as json_file:
    evts = json.load(json_file)

In [65]:
dfcom = pd.DataFrame(coms)
dfevt = pd.DataFrame(evts)

### Nombre de communes actuelles

Total

In [66]:
dfcom[dfcom['evtSuppression'].isna()].shape

(37697, 7)

Par type

In [67]:
dfcom[dfcom['evtSuppression'].isna()]['type'].value_counts()

COM     34965
COMD     2215
COMA      517
Name: type, dtype: int64

Aucune commune sans événement de création

In [68]:
dfcom[dfcom['evtCreation'].isna()]

,codeInsee,nom,nomSansArticle,evtCreation,id,type,evtSuppression


Comparaison fichier csv COG datagouv et données sparql

In [69]:
df = pd.read_csv('commune2021.csv')

In [70]:
df[(df['TYPECOM'] == 'COMA') & (df['COM'] == '19267')]

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
7092,COMA,19267,NaN,NaN,NaN,NaN,3,TOURETTE,Tourette,La Tourette,NaN,19275.0


Filtre hors ARM : arrondissement

In [71]:
listcog = df[df['TYPECOM'] != 'ARM']['COM'].unique().tolist()

Filtre commune qui ont eu une suppression pour ne garder que les actuelles

In [72]:
listrdf = dfcom[dfcom['evtSuppression'].isna()]['codeInsee'].unique().tolist()

Comparaison des deux listes

In [73]:
list(set(listcog) - set(listrdf))

[]

In [74]:
len(listcog)

37077

In [75]:
len(listrdf)

37077

=> On est bon

### Fonctions

Fonctions récupération evt

In [76]:
def getEvtByCodeInsee(codeInsee):
    list_coms = dfcom[dfcom['codeInsee'] == codeInsee].id.to_list()
    print(list_coms)
    dftest = dfevt
    dftest['is_create'] = dftest['creation'].apply(lambda x: not set(list_coms).isdisjoint(x))
    df1 = dftest[dftest['is_create'] == True]
    dftest['is_supp'] = dftest['suppression'].apply(lambda x: not set(list_coms).isdisjoint(x))
    df2 = dftest[dftest['is_supp'] == True]
    df3 = pd.concat([df1,df2])
    return df3[~df3.index.duplicated(keep='first')].sort_values(by=['date'])

In [77]:
def getEvtByNom(nom):
    list_coms = dfcom[dfcom['nom'] == nom].id.to_list()
    print(list_coms)
    dftest = dfevt
    dftest['is_create'] = dftest['creation'].apply(lambda x: not set(list_coms).isdisjoint(x))
    df1 = dftest[dftest['is_create'] == True]
    dftest['is_supp'] = dftest['suppression'].apply(lambda x: not set(list_coms).isdisjoint(x))
    df2 = dftest[dftest['is_supp'] == True]
    df3 = pd.concat([df1,df2])
    return df3[~df3.index.duplicated(keep='first')].sort_values(by=['date'])

In [78]:
getEvtByCodeInsee('27032')

['d171728d-5c4a-48a7-83c2-f62e8db01f99', 'a0dc41c9-8026-4382-9f5c-5b4075ab90e8', 'bfb5f656-4440-4561-bb49-e2136b9b7803']


,creation,suppression,date,id,type,creationObj,suppressionObj,is_create,is_supp
4469,"[4c09a74f-4b9d-4c2c-9e03-5574c979aa97, 0ce410a...",[],1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,evenement,"[{'codeInsee': '32019', 'nom': 'Auterive', 'no...",[],True,NaN
563,"[d171728d-5c4a-48a7-83c2-f62e8db01f99, bfb5f65...","[a1ee0f67-b657-418d-822b-7eaab68a89eb, 826d072...",2016-01-01,40ae74a8-6433-4dfa-a388-c98b629954d5,evenement,"[{'codeInsee': '27032', 'nom': 'Chambois', 'no...","[{'codeInsee': '27634', 'nom': 'Thomer-la-Sôgn...",True,NaN
5060,[],"[bfb5f656-4440-4561-bb49-e2136b9b7803, 9bd0400...",2020-12-31,4d54af6e-0a1a-4b4f-970b-3aa95d51e21e,evenement,[],"[{'codeInsee': '27032', 'nom': 'Avrilly', 'nom...",False,True


In [79]:
getEvtByNom('Collonges-lès-Premières')

['44168da9-25d9-4dc0-afdc-46b68c6e32a9', '7b036817-1be0-4b7a-9c47-7dfeb0486f5c']


,creation,suppression,date,id,type,creationObj,suppressionObj,is_create,is_supp
4469,"[4c09a74f-4b9d-4c2c-9e03-5574c979aa97, 0ce410a...",[],1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,evenement,"[{'codeInsee': '32019', 'nom': 'Auterive', 'no...",[],True,False
48,"[0376f0f2-9924-41e6-aaee-43542abdda70, 7b03681...","[44168da9-25d9-4dc0-afdc-46b68c6e32a9, 1a7ae1d...",2019-02-28,ff7e44ab-9e34-4152-8008-fdc8b6b34614,evenement,"[{'codeInsee': '21183', 'nom': 'Collonges-et-P...","[{'codeInsee': '21183', 'nom': 'Collonges-lès-...",True,False


Analyse nombre création / suppression par événement

In [80]:
dfevt['nb_creation'] = dfevt['creation'].apply(lambda x: len(x))
dfevt['nb_suppression'] = dfevt['suppression'].apply(lambda x: len(x))

Combien d'événement ont concerné plus d'une commune en création et plus d'une commune en suppression :

In [81]:
dfevt.shape

(5086, 11)

In [82]:
dfevt[(dfevt['nb_creation'] > 1) & (dfevt['nb_suppression'] > 1)].shape[0]

1798

In [83]:
dfevt.nb_creation.value_counts()

1        3104
2         832
3         591
4         229
5          93
0          90
6          43
7          27
8          25
10         10
9           7
11          7
15          6
12          4
16          3
14          2
19          2
343         1
2467        1
306         1
27          1
38125       1
94          1
17          1
20          1
23          1
21          1
699         1
Name: nb_creation, dtype: int64

In [84]:
dfevt.nb_suppression.value_counts()

1     2383
2     1833
3      454
4      188
5       69
6       44
0       39
7       21
8       14
9       11
10      11
14       5
22       2
11       2
17       2
15       2
12       2
21       1
19       1
18       1
16       1
Name: nb_suppression, dtype: int64

Combien d'événement ont impliqués plus de communes créées que de communes supprimées (tout type de commune confondu) :

In [85]:
dfevt[(dfevt['nb_creation'] > 1) & (dfevt['nb_suppression'] > 1) & (dfevt['nb_creation'] >= dfevt['nb_suppression']+1)].shape[0]

657

On à toujours au max qu'une seule création en plus que de suppression :

In [86]:
dfevt[(dfevt['nb_creation'] > 1) & (dfevt['nb_suppression'] > 1) & (dfevt['nb_creation'] >= dfevt['nb_suppression']+2)].shape[0]

0

On va rechercher les événements qui ont créé plus de commune COM que de suppression de COM :
=> si il en existe, cela veut dire qu'on ne peut pas garantir l'intégrité territoriale dans le temps sur certaines communes

In [87]:
dftest = dfevt[(dfevt['nb_creation'] > 1) & (dfevt['nb_suppression'] > 1) & (dfevt['nb_creation'] >= dfevt['nb_suppression']+1)]

In [88]:
arr = []
for index, row in dftest.iterrows():
    nb_com_in = 0
    nb_coma_in = 0
    nb_comd_in = 0
    nb_com_out = 0
    nb_coma_out = 0
    nb_comd_out = 0
    for com in row['creationObj']:
        if(com['type'] == 'COM'):
            nb_com_out = nb_com_out + 1
        if(com['type'] == 'COMD'):
            nb_comd_out = nb_comd_out + 1
        if(com['type'] == 'COMA'):
            nb_coma_out = nb_coma_out + 1
    for com in row['suppressionObj']:
        if(com['type'] == 'COM'):
            nb_com_in = nb_com_in + 1
        if(com['type'] == 'COMD'):
            nb_comd_in = nb_comd_in + 1
        if(com['type'] == 'COMA'):
            nb_coma_in = nb_coma_in + 1
    if(nb_com_out > 1):
        arr.append(row)
        print('---------')
        print('Evenement '+row['id']+' : ')
        print('Suppression : '+str(nb_com_in)+' communes, '+str(nb_comd_in)+' communes déléguées, '+str(nb_coma_in)+' communes associées')
        print('Creation : '+str(nb_com_out)+' communes, '+str(nb_comd_out)+' communes déléguées, '+str(nb_coma_out)+' communes associées')

dfout = pd.DataFrame(arr)

---------
Evenement 637f4e23-fb2d-4f95-8bd6-36c78c699b3b : 
Suppression : 3 communes, 0 communes déléguées, 0 communes associées
Creation : 4 communes, 0 communes déléguées, 0 communes associées
---------
Evenement 38377343-88a6-458b-8142-162370e38638 : 
Suppression : 2 communes, 0 communes déléguées, 0 communes associées
Creation : 3 communes, 0 communes déléguées, 0 communes associées
---------
Evenement 2c4e2074-25ce-40a1-b7d3-39e28a238982 : 
Suppression : 4 communes, 0 communes déléguées, 0 communes associées
Creation : 5 communes, 0 communes déléguées, 0 communes associées
---------
Evenement 662cfa72-c4e3-49d1-94bb-7ee067c3a726 : 
Suppression : 8 communes, 0 communes déléguées, 0 communes associées
Creation : 9 communes, 0 communes déléguées, 0 communes associées
---------
Evenement 2cc22e41-441b-4897-ad76-22fc1cd17415 : 
Suppression : 2 communes, 0 communes déléguées, 0 communes associées
Creation : 3 communes, 0 communes déléguées, 0 communes associées
---------
Evenement dff90

In [89]:
for index, row in dfout.iterrows():
    print('-------')
    mystr = ''
    for com in row['suppressionObj']:
        mystr = mystr+com['nom']+' ('+com['codeInsee']+' - '+com['type']+') ; '
    
    mystr = mystr + '\ndeviennent\n'
                                          
    for com in row['creationObj']:
        mystr = mystr+com['nom']+' ('+com['codeInsee']+' - '+com['type']+') ; '
    print(mystr)

-------
Vaulnaveys-le-Haut (38529 - COM) ; Séchilienne (38478 - COM) ; Saint-Martin-d'Uriage (38422 - COM) ; 
deviennent
Chamrousse (38567 - COM) ; Vaulnaveys-le-Haut (38529 - COM) ; Séchilienne (38478 - COM) ; Saint-Martin-d'Uriage (38422 - COM) ; 
-------
Bures-sur-Yvette (91122 - COM) ; Orsay (91471 - COM) ; 
deviennent
Bures-sur-Yvette (91122 - COM) ; Les Ulis (91692 - COM) ; Orsay (91471 - COM) ; 
-------
Saint-Jean-Lespinasse (46271 - COM) ; Saint-Vincent-du-Pendit (46295 - COM) ; Saint-Médard-de-Presque (46281 - COM) ; Saint-Céré (46251 - COM) ; 
deviennent
Saint-Médard-de-Presque (46281 - COM) ; Saint-Vincent-du-Pendit (46295 - COM) ; Saint-Jean-Lespinasse (46271 - COM) ; Saint-Jean-Lagineste (46339 - COM) ; Saint-Céré (46251 - COM) ; 
-------
Porte-Joie (27471 - COM) ; Poses (27474 - COM) ; Tournedos-sur-Seine (27651 - COM) ; Léry (27365 - COM) ; Saint-Pierre-du-Vauvray (27598 - COM) ; Incarville (27351 - COM) ; Saint-Étienne-du-Vauvray (27537 - COM) ; Le Vaudreuil (27528 - CO

In [90]:
arr = []
for index, row in dftest.iterrows():
    nb_com_in = 0
    nb_coma_in = 0
    nb_comd_in = 0
    nb_com_out = 0
    nb_coma_out = 0
    nb_comd_out = 0
    for com in row['creationObj']:
        if(com['type'] == 'COM'):
            nb_com_out = nb_com_out + 1
        if(com['type'] == 'COMD'):
            nb_comd_out = nb_comd_out + 1
        if(com['type'] == 'COMA'):
            nb_coma_out = nb_coma_out + 1
    for com in row['suppressionObj']:
        if(com['type'] == 'COM'):
            nb_com_in = nb_com_in + 1
        if(com['type'] == 'COMD'):
            nb_comd_in = nb_comd_in + 1
        if(com['type'] == 'COMA'):
            nb_coma_in = nb_coma_in + 1
    if(nb_coma_out > 1):
        arr.append(row)
        print('---------')
        print('Evenement '+row['id']+' : ')
        print('Suppression : '+str(nb_com_in)+' communes, '+str(nb_comd_in)+' communes déléguées, '+str(nb_coma_in)+' communes associées')
        print('Creation : '+str(nb_com_out)+' communes, '+str(nb_comd_out)+' communes déléguées, '+str(nb_coma_out)+' communes associées')

dfout = pd.DataFrame(arr)

In [91]:
for index, row in dfout.iterrows():
    print('-------')
    mystr = ''
    for com in row['suppressionObj']:
        mystr = mystr+com['nom']+' ('+com['codeInsee']+' - '+com['type']+') ; '
    
    mystr = mystr + '\ndeviennent\n'
                                          
    for com in row['creationObj']:
        mystr = mystr+com['nom']+' ('+com['codeInsee']+' - '+com['type']+') ; '
    print(mystr)

Evenement qui ont créé plus de 2 communes COM

In [92]:
arr = []
for index, row in dfevt.iterrows():
    nb_com = 0
    for com in row['creationObj']:
        if(com['type'] == 'COM'):
            nb_com = nb_com + 1
    if(nb_com > 1):
        arr.append(row)
dfout = pd.DataFrame(arr)

In [93]:
dfout.shape[0]

274

In [94]:
for index, row in dfout.iterrows():
    comcreate = [x['nomSansArticle'] for x in row['creationObj']]
    comsupp = [x['nomSansArticle'] for x in row['suppressionObj']]

    #print(comcreate)
    #print(comsupp)
    #print('---')


In [95]:
df = dfevt.sort_values(by=['date'])
df = df[~((df['creationObj'].str.len() == 0) & (df['suppressionObj'].str.len() == 0))]

In [96]:
df.head()

,creation,suppression,date,id,type,creationObj,suppressionObj,is_create,is_supp,nb_creation,nb_suppression
2549,[3d13fae4-16d2-4e96-bd91-fbaeb688dd3c],"[bc622910-0e35-4b04-aab8-30d3ecf6d013, def17c4...",1942-08-01,2ffc80b6-b246-4d33-b6a9-46b711fe5de0,evenement,"[{'codeInsee': '01185', 'nom': 'Hauteville-Lom...","[{'codeInsee': '01220', 'nom': 'Lompnes', 'nom...",False,False,1,2
3158,[cbdb7b26-3259-4ac1-b302-1bc10a4711a6],"[76c37d0f-88fd-47b3-9674-000b0e667910, 9fb6ef3...",1942-10-01,6ac12379-99f0-43c6-afe7-83aa0ed22f4b,evenement,"[{'codeInsee': '54518', 'nom': 'Thiaucourt', '...","[{'codeInsee': '54448', 'nom': 'Regniéville', ...",False,False,1,2
2827,[6d3b1c48-cd95-4f57-ab97-9241d2da5e17],"[d50ff06d-77a1-42e9-bde5-b2a612b89919, 4aa381a...",1942-10-01,5dd8818d-0eb5-4326-866d-99076e689233,evenement,"[{'codeInsee': '54316', 'nom': 'Limey', 'nomSa...","[{'codeInsee': '54454', 'nom': 'Remenauville',...",False,False,1,2
3027,[8fcf9222-0527-4733-b33f-e8c7735adc07],"[2830f25b-ab37-48f3-8afb-6f20075c613f, cf2860c...",1942-11-01,065710af-e6d6-4a3d-83d4-b2ce7bbd7b0d,evenement,"[{'codeInsee': '66003', 'nom': 'Amélie-les-Bai...","[{'codeInsee': '66003', 'nom': 'Amélie-les-Bai...",False,False,1,2
4469,"[4c09a74f-4b9d-4c2c-9e03-5574c979aa97, 0ce410a...",[],1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,evenement,"[{'codeInsee': '32019', 'nom': 'Auterive', 'no...",[],True,False,38125,0


In [97]:
arr = []
cpt = 0
for index, row in df.iterrows():
    # 1 ou N création, 0 suppression
    if((len(row['creationObj']) >= 1) & (len(row['suppressionObj']) == 0)):
        for com in row['creationObj']:
            mydict = {}
            mydict['nom'] = com['nom']
            mydict['type'] = com['type']
            mydict['dateDebut'] = row['date']
            mydict['code'] = com['codeInsee']
            mydict['predecesseur'] = None
            mydict['id'] = com['type'] + '-' + com['codeInsee'] + '@' + row['date']
            mydict['idEvt'] = row['id']
            arr.append(mydict)
            
    # 1 ou N création, 1 suppression
    if((len(row['creationObj']) >= 1) & (len(row['suppressionObj']) == 1)):
        for com in row['creationObj']:
            mydict = {}
            mydict['nom'] = com['nom']
            mydict['type'] = com['type']
            mydict['dateDebut'] = row['date']
            mydict['code'] = com['codeInsee']
            mydict['predecesseur'] = row['suppressionObj'][0]['type'] + '-' + row['suppressionObj'][0]['codeInsee'] + '@' + df[df['id'] == row['suppressionObj'][0]['evtCreation']].iloc[0]['date']
            mydict['id'] = com['type'] + '-' + com['codeInsee'] + '@' + row['date']
            mydict['idEvt'] = row['id']
            arr.append(mydict)
            
    # N création, N suppression
    if((len(row['creationObj']) >= 1) & (len(row['suppressionObj']) > 1)):
        nbcom = 0
        codecom = ''
        datecom = ''
        for com in row['suppressionObj']:
            if(com['type'] == 'COM'):
                nbcom += 1
                codecom = com['codeInsee']
                datecom = df[df['id'] == com['evtCreation']].iloc[0]['date']
        if(nbcom > 1):
            for com in row['creationObj']:
                mydict = {}
                mydict['nom'] = com['nom']
                mydict['type'] = com['type']
                mydict['dateDebut'] = row['date']
                mydict['code'] = com['codeInsee']
                codesInsee = []
                for com2 in row['suppressionObj']:
                    codesInsee.append(com2['codeInsee'])
                codesInsee = list(dict.fromkeys(codesInsee))
                mydict['predecesseur'] = 'COMT-' + '-'.join(codesInsee) + '@' + row['date']
                mydict['id'] = com['type'] + '-' + com['codeInsee'] + '@' + row['date']
                mydict['idEvt'] = row['id']
                arr.append(mydict)
        elif(nbcom == 1):
            for com in row['creationObj']:
                mydict = {}
                mydict['nom'] = com['nom']
                mydict['type'] = com['type']
                mydict['dateDebut'] = row['date']
                mydict['code'] = com['codeInsee']
                mydict['predecesseur'] = 'COM-' + codecom + '@' + datecom
                mydict['id'] = com['type'] + '-' + com['codeInsee'] + '@' + row['date']
                mydict['idEvt'] = row['id']
                arr.append(mydict)
        else:
            print('pb en perspective - 0 commune supprimée mais des comd et coma')

    # 0 création, N suppression
print(cpt)
    
    

0


In [98]:
len(arr)

46782

In [99]:
toto = pd.DataFrame(arr)
toto['successeur'] = ''

In [153]:
arr = []
cpt = 0
for index, row in df.iterrows():
    
    # 1 ou N suppression, 0 création
    if((len(row['suppressionObj']) >= 1) & (len(row['creationObj']) == 0)):
        for com in row['suppressionObj']:
            mydict = {}
            mydict['nom'] = com['nom']
            mydict['type'] = 'COMP'
            mydict['dateDebut'] = row['date']
            mydict['code'] = com['codeInsee']
            mydict['predecesseur'] = com['type'] + '-' + com['codeInsee'] + '@' + toto[toto['idEvt'] == com['evtCreation']].iloc[0]['dateDebut']
            mydict['id'] = 'COMP-' + com['codeInsee'] + '@' + row['date']
            mydict['idEvt'] = row['id']
            arr.append(mydict) 
            
            mydate = toto[(toto['dateDebut'] <= row['date']) & (toto['id'].str.contains(com['type']+'-'+com['codeInsee']))].sort_values(by='dateDebut',ascending=False).iloc[0]['dateDebut']
            toto.loc[(toto['id'] == com['type']+'-'+com['codeInsee']+'@'+mydate),'successeur'] = 'COMP-' + com['codeInsee'] + '@' + row['date']
            
            
    # N suppression, N creation
    if((len(row['suppressionObj']) > 1) & (len(row['creationObj']) >= 1)):
        cpt = cpt + 1
        if(cpt % 200 == 0):
            print(cpt)
        codesInsee = []
        for com in row['suppressionObj']:
            if(com['type'] == 'COM'):
                codesInsee.append(com['codeInsee'])
        codesInsee = list(dict.fromkeys(codesInsee))
        mydict = {}
        mydict['nom'] = 'Commune Transitoire'
        mydict['type'] = 'COMT'
        mydict['dateDebut'] = row['date']
        mydict['code'] = '-'.join(codesInsee)
        mydict['id'] = 'COMT-' + '-'.join(codesInsee) + '@' + row['date']
        mydict['idEvt'] = row['id']
        mydict['predecesseur'] = None
        arr.append(mydict)
        for com in row['suppressionObj']:
            try:
                mydate = toto[(toto['dateDebut'] <= row['date']) & (toto['id'].str.contains(com['type']+'-'+com['codeInsee']))].sort_values(by='dateDebut',ascending=False).iloc[0]['dateDebut']
                toto.loc[(toto['id'] == com['type']+'-'+com['codeInsee']+'@'+mydate),'successeur'] = 'COMT-' + '-'.join(codesInsee) + '@' + row['date']
            except:
                print('Pb intégrité base SPARQL - id Evt INSEE : '+row['id'])
    
    if((len(row['suppressionObj']) == 1) & (len(row['creationObj']) == 1)):
        mydate = toto[(toto['dateDebut'] <= row['date']) & (toto['id'].str.contains(row['suppressionObj'][0]['type']+'-'+row['suppressionObj'][0]['codeInsee']))].sort_values(by='dateDebut',ascending=False).iloc[0]['dateDebut']
        toto.loc[(toto['id'] == row['suppressionObj'][0]['type']+'-'+row['suppressionObj'][0]['codeInsee']+'@'+mydate),'successeur'] = row['creationObj'][0]['type'] + '-' + row['creationObj'][0]['codeInsee'] + '@' + row['date']
        
    
print(cpt)


Pb intégrité base SPARQL - id Evt INSEE : 2ffc80b6-b246-4d33-b6a9-46b711fe5de0
Pb intégrité base SPARQL - id Evt INSEE : 6ac12379-99f0-43c6-afe7-83aa0ed22f4b
Pb intégrité base SPARQL - id Evt INSEE : 5dd8818d-0eb5-4326-866d-99076e689233
Pb intégrité base SPARQL - id Evt INSEE : 065710af-e6d6-4a3d-83d4-b2ce7bbd7b0d
200
creation                     [002ffef9-dee5-47b3-8fd2-920176692b5b]
suppression                  [6cb8d621-e597-48ba-a6eb-dbf0bf2141f6]
date                                                     1968-01-01
id                             8dba2b23-ea40-4cd8-b9cd-32eb19e4770b
type                                                      evenement
creationObj       [{'codeInsee': '91639', 'nom': 'Vayres-sur-Ess...
suppressionObj    [{'codeInsee': '78639', 'nom': 'Vayres-sur-Ess...
is_create                                                     False
is_supp                                                       False
nb_creation                                                       1


In [154]:
tutu = pd.DataFrame(arr)

In [155]:
titi = pd.concat([toto,tutu])

In [156]:
titi['successeur'] = titi['successeur'].apply(lambda x: None if x == '' else x)

In [157]:
titi[titi['successeur'].isna()].shape

(38660, 8)

In [158]:
cog = pd.read_csv('commune2021.csv')

In [159]:
titi[titi['nom'] == 'Vayres-sur-Essonne']

,nom,type,dateDebut,code,predecesseur,id,idEvt,successeur
6,Vayres-sur-Essonne,COM,1943-01-01,78639,None,COM-78639@1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,COM-91639@1968-01-01
39690,Vayres-sur-Essonne,COM,1968-01-01,91639,COM-78639@1943-01-01,COM-91639@1968-01-01,8dba2b23-ea40-4cd8-b9cd-32eb19e4770b,None


In [161]:
for index, row in titi.iterrows():
    if(row['successeur'] is None):
        if(cog[(cog['TYPECOM'] == row['type']) & (cog['COM'] == row['code'])].shape[0] == 0):
            print(row)

nom                                         Monceaux
type                                             COM
dateDebut                                 1943-01-01
code                                           61280
predecesseur                                    None
id                              COM-61280@1943-01-01
idEvt           904dcf14-b895-4686-a5c5-7304c6f63cae
successeur                                      None
Name: 362, dtype: object
nom                                     Saint-Martin
type                                             COM
dateDebut                                 1943-01-01
code                                           52451
predecesseur                                    None
id                              COM-52451@1943-01-01
idEvt           904dcf14-b895-4686-a5c5-7304c6f63cae
successeur                                      None
Name: 853, dtype: object
nom                                           Aulhat
type                                             

nom                                             Vaux
type                                             COM
dateDebut                                 1943-01-01
code                                           25593
predecesseur                                    None
id                              COM-25593@1943-01-01
idEvt           904dcf14-b895-4686-a5c5-7304c6f63cae
successeur                                      None
Name: 4774, dtype: object
nom                                            Ceaux
type                                             COM
dateDebut                                 1943-01-01
code                                           86043
predecesseur                                    None
id                              COM-86043@1943-01-01
idEvt           904dcf14-b895-4686-a5c5-7304c6f63cae
successeur                                      None
Name: 4839, dtype: object
nom                                          Bagnols
type                                           

nom                                      Coudekerque
type                                             COM
dateDebut                                 1943-01-01
code                                           59154
predecesseur                                    None
id                              COM-59154@1943-01-01
idEvt           904dcf14-b895-4686-a5c5-7304c6f63cae
successeur                                      None
Name: 8431, dtype: object
nom                                            Bures
type                                             COM
dateDebut                                 1943-01-01
code                                           14114
predecesseur                                    None
id                              COM-14114@1943-01-01
idEvt           904dcf14-b895-4686-a5c5-7304c6f63cae
successeur                                      None
Name: 8454, dtype: object
nom                                    Saint-Germain
type                                           

KeyboardInterrupt: 

In [128]:
cog[cog['COM'] == '78639']

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT


In [165]:
titi[titi['code'] == '61280']

,nom,type,dateDebut,code,predecesseur,id,idEvt,successeur
362,Monceaux,COM,1943-01-01,61280,None,COM-61280@1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,None
43006,Monceaux-au-Perche,COM,1992-09-01,61280,COM-61280@1943-01-01,COM-61280@1992-09-01,a779b151-7118-4fac-bd06-16818294a47b,COMT-61280-61247-61458-61250-61220-61305-61296...
43755,Monceaux-au-Perche,COMD,2016-01-01,61280,COMT-61280-61247-61458-61250-61220-61305-61296...,COMD-61280@2016-01-01,5dec92fc-40b7-4714-8dcd-c2c64dd0cb1c,None


In [168]:
for index, row in titi.iterrows():
    if(row['successeur'] is None):
        if(titi[titi['predecesseur'] == row['id']].shape[0] > 0):
            row['successeur'] = titi[titi['predecesseur'] == row['id']].iloc[0]['id']

In [214]:
for index, row in titi.iterrows():
    if(row['id'] == row['successeur']):
        row['successeur'] = None

In [169]:
titi[titi['code'] == '61280']

,nom,type,dateDebut,code,predecesseur,id,idEvt,successeur
362,Monceaux,COM,1943-01-01,61280,None,COM-61280@1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,COM-61280@1992-09-01
43006,Monceaux-au-Perche,COM,1992-09-01,61280,COM-61280@1943-01-01,COM-61280@1992-09-01,a779b151-7118-4fac-bd06-16818294a47b,COMT-61280-61247-61458-61250-61220-61305-61296...
43755,Monceaux-au-Perche,COMD,2016-01-01,61280,COMT-61280-61247-61458-61250-61220-61305-61296...,COMD-61280@2016-01-01,5dec92fc-40b7-4714-8dcd-c2c64dd0cb1c,None


In [174]:
cog = cog[cog['TYPECOM'] != 'ARM']

In [215]:
out = titi.copy()

In [202]:
out = out[out['successeur'].isna()]

In [208]:
cpt = 0
for index, row in out.iterrows():
    cpt = cpt + 1
    if(cpt % 1000 == 0):
        print(cpt)
    if((row['type'] != 'COMT') & (row['type'] != 'COMP')):
        if(cog[(cog['TYPECOM'] == row['type']) & (cog['COM'] == row['code'])].shape[0] == 0):
            print(row)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
nom                                      Regniéville
type                                             COM
dateDebut                                 1943-01-01
code                                           54448
predecesseur                                    None
id                              COM-54448@1943-01-01
idEvt           904dcf14-b895-4686-a5c5-7304c6f63cae
successeur                                      None
Name: 15802, dtype: object
12000
13000
14000
15000
16000
nom                                          Lompnes
type                                             COM
dateDebut                                 1943-01-01
code                                           01220
predecesseur                                    None
id                              COM-01220@1943-01-01
idEvt           904dcf14-b895-4686-a5c5-7304c6f63cae
successeur                                      None
Name: 21762, dtype: object
17000
nom  

In [216]:
cpt = 0
for index, row in cog.iterrows():
    cpt = cpt + 1
    if(cpt % 1000 == 0):
        print(cpt)
    if(out[(out['type'] == row['TYPECOM']) & (out['code'] == row['COM'])].shape[0] == 0):
        print(row)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


In [217]:
import json
with open('graph-coms.json', 'w') as f:
    json.dump(titi.to_dict(orient="records"), f, indent=4)

In [218]:
cog

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84.0,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
37737,COM,97613,6.0,976,976D,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN
37738,COM,97614,6.0,976,976D,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN
37739,COM,97615,6.0,976,976D,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN
37740,COM,97616,6.0,976,976D,NaN,0,SADA,Sada,Sada,97612,NaN


In [222]:
cog[cog['NCCENR'] == 'Boissy-sur-Damville']

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT


In [223]:
titi[titi['code'] == '27080']

,nom,type,dateDebut,code,predecesseur,id,idEvt,successeur
2656,Boissy-sur-Damville,COM,1943-01-01,27080,None,COM-27080@1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,COMT-27416-27186-27080@1972-01-01
40114,Boissy-sur-Damville,COMA,1972-01-01,27080,COMT-27416-27186-27080@1972-01-01,COMA-27080@1972-01-01,b2193291-9d40-45dd-8e50-8d01a552aa24,COMT-27416@1978-07-01


In [225]:
titi[titi['code'].str.contains() == '27416']

TypeError: contains() missing 1 required positional argument: 'pat'

Cas particulier:
- Hauteville https://rdf.insee.fr/sparql/describe?uri=http%3A%2F%2Fid.insee.fr%2Fgeo%2Fcommune%2Fdef17c47-5128-401d-9e42-0690d8c0daae&default-graph-uri=data
- Créé en 1943, supprimé en 1942....
    